# Spark 101 Exercises
### Kwame V. Taylor

1. Create a spark data frame that contains your favorite programming languages.
  * The name of the column should be ```language```
  * View the schema of the dataframe
  * Output the shape of the dataframe
  * Show the first 5 records in the dataframe

**Imports and start Spark session**

In [29]:
import pandas as pd
import numpy as np

np.random.seed(666)

import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import col, exp, lit, concat, regexp_extract, regexp_replace, expr
from pyspark.sql.functions import round, sum, avg, min, max, count, mean
from pyspark.sql.functions import udf, year, month, quarter, asc, desc
from pyspark.sql.types import FloatType

spark = pyspark.sql.SparkSession.builder.getOrCreate()

**Create custom pyspark shape function**

In [13]:
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape

**Create Spark dataframe**

In [2]:
# Create pandas dataframe by columns using dictionary-like object
pd_df = pd.DataFrame({'language': ['Python', 'Java', 'HTML', 'CSS', 'JavaScript']}, 
                     index = [1, 2, 3, 4, 5])
pd_df

,language
1,Python
2,Java
3,HTML
4,CSS
5,JavaScript


In [4]:
# Convert pandas dataframe to spark dataframe
df = spark.createDataFrame(pd_df)
df

DataFrame[language: string]

**View schema of dataframe**

In [8]:
df.printSchema()

root
 |-- language: string (nullable = true)



**Print shape of dataframe**

In [15]:
print((df.count(), len(df.columns)))

(5, 1)


Alternatively, I can use my custom pyspark shape function.

In [14]:
df.shape()

(5, 1)

**Print first 5 records**

In [5]:
df.show(5)

+----------+
|  language|
+----------+
|    Python|
|      Java|
|      HTML|
|       CSS|
|JavaScript|
+----------+



**Describe the dataframe**

In [16]:
df.describe().show()

+-------+--------+
|summary|language|
+-------+--------+
|  count|       5|
|   mean|    null|
| stddev|    null|
|    min|     CSS|
|    max|  Python|
+-------+--------+



2. Load the ```mpg``` dataset as a spark dataframe.

    a. Create 1 column of output that contains a message like the one below:
        The 1999 audi a4 has a 4 cylinder engine.
    For each vehicle.
    
    b. Transform the ```trans``` column so that it only contains either ```manual``` or ```auto```.

**Load mpg dataset**

In [17]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



**Create output column**

In [33]:
# Create an output column for information about each vechicle in mpg
mpg = (mpg.withColumn(colName='output',
                      col=concat(lit('The '), 'year', lit(' '), 'manufacturer', lit(' '),
                                 'model', lit(' has a '), 'cyl', lit(' cylinder engine.'))))

In [40]:
mpg.select('output').show(truncate=False)

+--------------------------------------------------------------+
|output                                                        |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

**Transform trans column**

In [42]:
# Use regex to transform the trans column so that it only contains either manual or auto.

mpg.select(regexp_replace("trans", r"\([^)]*\)", "").alias("transonly")).show(5)

+---------+
|transonly|
+---------+
|     auto|
|   manual|
|   manual|
|     auto|
|     auto|
+---------+
only showing top 5 rows



3. Load the ```tips``` dataset as a spark dataframe.

    a. What percentage of observations are smokers?
    
    b. Create a column that contains the tip percentage
    
    c. Calculate the average tip percentage for each combination of sex and smoker.

**Load tips dataset**

In [43]:
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



**What percentage of observations are smokers?**

In [50]:
percent_smoke = tips.groupBy('smoker').agg(round(count(tips.smoker)/ tips.count(),2).alias("percent"))

In [51]:
percent_smoke.show()

+------+-------+
|smoker|percent|
+------+-------+
|    No|   0.62|
|   Yes|   0.38|
+------+-------+



38% of observations are smokers.

**Create a column that contains the tip percentage**

In [67]:
tips = tips.withColumn(colName='tip_percentage', col=round(col('tip') / col('total_bill'), 2))
tips.show(10)

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|          0.06|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          0.16|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          0.17|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          0.14|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          0.15|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|          0.19|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|          0.23|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|          0.12|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|          0.13|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|          0.22|
+----------+----+------+------+---+------+----+--------------+
only showing top 10 rows



**Calculate the average tip percentage for each combination of sex and smoker.**

In [76]:
avg_tip = tips.groupBy('smoker', 'sex').agg(round(avg('tip_percentage'),4).alias("average_tip")).sort('average_tip', ascending=False)

In [77]:
avg_tip.show()

+------+------+-----------+
|smoker|   sex|average_tip|
+------+------+-----------+
|   Yes|Female|     0.1824|
|    No|  Male|      0.161|
|    No|Female|     0.1569|
|   Yes|  Male|     0.1528|
+------+------+-----------+



It seems that smoking females tip the most, based on this data.